In [1]:
import pandas as pd
import numpy as np
import datetime
import os
import glob
os.getcwd()

'/home/jian/Projects/Big_Lots/Analysis/2019_Q3/Planner_request/Bruce_competitors_in_PS_zips_20190830'

In [2]:
list_competitors=pd.ExcelFile("./BL Sito Raw and Output sample file 20190528 BR.xlsx")
list_competitors=list_competitors.parse("Input RAW Both combined visits ",dtype=str)
list_competitors=list_competitors["brand"].unique().tolist()
list_competitors=[x for x in list_competitors if x!="nan"]
print(len(list_competitors))
print(list_competitors)

7
['Walmart Supercenter', 'Dollar General', 'Dollar Tree', "Kohl's", 'Target', "Bob's Discount Furniture", 'Walmart']


In [3]:
zips_BL_PS=pd.ExcelFile("/home/jian/Projects/Big_Lots/New_TA/TA_created_in_201906/final_output_20190718/BL_final_TA_updated_remove_8_closing_stores_JL_2019-08-20.xlsx")
print(zips_BL_PS.sheet_names)
zips_BL_PS=zips_BL_PS.parse("unique_zips_full_footprint",dtype=str)
print(zips_BL_PS['zip_type'].unique())
zips_BL_PS.head(2)

['unique_zips_full_footprint', 'view_by_TA', 'view_by_store']
['trans_P' 'trans_S' 'zips_10']


,zip_type,zip_cd
0,trans_P,17403
1,trans_P,41522


In [4]:
latest_SG_brand_info=pd.read_csv("/home/jian/Docs/SafeGraph/places_20190629/brand_info/part-00000-tid-3844431843075532209-d890661b-f976-44e8-9678-725e8cd08c6f-119-c000.csv.gz",
                                dtype=str,compression="gzip",sep="|")

latest_SG_brand_info=latest_SG_brand_info[latest_SG_brand_info["brand_name"].isin(list_competitors)]
print(latest_SG_brand_info.shape)
latest_SG_brand_info


(6, 6)


,safegraph_brand_id,brand_name,parent_safegraph_brand_id,naics_code,top_category,sub_category
566,SG_BRAND_11126e8062018d7b6b1ede4745827e72,Bob's Discount Furniture,NaN,442110,Furniture Stores,Furniture Stores
1284,SG_BRAND_a4efb9d8ddf0f71207d49f713c5bc1e9,Dollar General,NaN,452319,"General Merchandise Stores, including Warehous...",All Other General Merchandise Stores
1287,SG_BRAND_6b1e006c89aa3f8b054a05fa90581501,Dollar Tree,NaN,452319,"General Merchandise Stores, including Warehous...",All Other General Merchandise Stores
2411,SG_BRAND_64a77880c7f7c1d3133d10e574c97a8b,Kohl's,NaN,452210,Department Stores,Department Stores
4138,SG_BRAND_42aefbae01d2dfd981f7da7d823d689e,Target,NaN,452210,Department Stores,Department Stores
4600,SG_BRAND_04a8ca7bf49e7ecb4a32451676e929f0,Walmart,SG_BRAND_f49758461e68a211,452319,"General Merchandise Stores, including Warehous...",All Other General Merchandise Stores


In [5]:
POI_data=glob.glob("/home/jian/Docs/SafeGraph/places_20190629/poi/*csv.gz")
# POI_data

In [6]:
store_dict_of_df={} 

for file in POI_data:
    df=pd.read_csv(file,dtype=str,compression="gzip",sep="|")
    for brand_name in list_competitors:
        df_brand_i=df[df['location_name']==brand_name]
        try:
            df_already_agg_brand=store_dict_of_df[brand_name]
            store_dict_of_df.update({brand_name:df_already_agg_brand.append(df_brand_i)})
        except:
            store_dict_of_df.update({brand_name:df_brand_i})

   
    print(datetime.datetime.now(),file)

2019-08-29 20:30:14.911316 /home/jian/Docs/SafeGraph/places_20190629/poi/part-00001-tid-9006125908719184114-1808d564-228a-44d4-8710-c00fc4c1b790-112-c000.csv.gz
2019-08-29 20:30:27.401250 /home/jian/Docs/SafeGraph/places_20190629/poi/part-00003-tid-9006125908719184114-1808d564-228a-44d4-8710-c00fc4c1b790-114-c000.csv.gz
2019-08-29 20:30:40.193537 /home/jian/Docs/SafeGraph/places_20190629/poi/part-00000-tid-9006125908719184114-1808d564-228a-44d4-8710-c00fc4c1b790-111-c000.csv.gz
2019-08-29 20:30:51.549659 /home/jian/Docs/SafeGraph/places_20190629/poi/part-00004-tid-9006125908719184114-1808d564-228a-44d4-8710-c00fc4c1b790-115-c000.csv.gz
2019-08-29 20:31:03.494741 /home/jian/Docs/SafeGraph/places_20190629/poi/part-00002-tid-9006125908719184114-1808d564-228a-44d4-8710-c00fc4c1b790-113-c000.csv.gz
2019-08-29 20:31:14.777690 /home/jian/Docs/SafeGraph/places_20190629/poi/part-00005-tid-9006125908719184114-1808d564-228a-44d4-8710-c00fc4c1b790-116-c000.csv.gz
2019-08-29 20:31:25.855737 /home/j

In [7]:
df_output=pd.DataFrame()
for k,v in store_dict_of_df.items():
    print(k,v.shape)
    df_output=df_output.append(v)

Walmart Supercenter (11, 20)
Dollar General (15032, 20)
Dollar Tree (6869, 20)
Kohl's (1133, 20)
Target (1834, 20)
Bob's Discount Furniture (113, 20)
Walmart (4615, 20)


In [8]:
print(df_output['zip_code'].apply(len).unique())
df_output['zip_code']=df_output['zip_code'].apply(lambda x: x.zfill(5))
print(df_output['zip_code'].apply(len).unique())


[5 4 3]
[5]


In [9]:
zips_BL_PS=zips_BL_PS.rename(columns={"zip_cd":"zip_code"})
df_output=pd.merge(df_output,zips_BL_PS,on="zip_code",how="left")
df_output_PS_only=df_output[pd.notnull(df_output['zip_type'])]

In [13]:
writer=pd.ExcelWriter("./BL_competitors_in_PS_zips_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_output_PS_only.to_excel(writer,"competitors_in_PS",index=False)
df_output.to_excel(writer,"all_competitors",index=False)
writer.save()